In [1]:
import geopandas as gpd
import pandas as pd
import requests

In [2]:
# https://data.cityofnewyork.us/Education/2021-DOE-Middle-School-Directory/f6s7-vytj/about_data
r = requests.get('https://data.cityofnewyork.us/resource/f6s7-vytj.json')
school_directory = r.json()
len(school_directory)

474

In [3]:
# source: https://data.cityofnewyork.us/Education/School-Point-Locations/jfju-ynrr/about_data
gdf = (
    gpd.read_file('source_data/SchoolPoints_APS_2024_08_28/SchoolPoints_APS_2024_08_28.shp')
    .rename(columns={'Geographic': 'District', 'ATS': 'DBN'})
    .set_index('DBN')
    .drop(columns=['Building_C', 'Location_C', 'Name'])
)
gdf.tail(5)

,District,Latitude,Longitude,geometry
DBN,,,,
88X966,8,40.816494,-73.890278,POINT (-8225428.122 4985312.134)
88X994,12,40.829306,-73.892243,POINT (-8225646.864 4987196.846)
88X995,10,40.857248,-73.903165,POINT (-8226862.696 4991308.524)
88X996,8,40.821218,-73.855930,POINT (-8221604.520 4986007.017)
93M359,2,40.747398,-73.992832,POINT (-8236844.381 4975154.021)


In [4]:
# https://infohub.nyced.org/reports/students-and-schools/school-quality/school-quality-reports-and-resources
excel_path = 'source_data/2022-23 NYC Schools Data.xlsx'
def read_sheet(sheet_name):
    return (
        pd.read_excel(
            excel_path,
            sheet_name=sheet_name
        ).melt(
            id_vars='DBN',
            var_name='Variable',
            value_name='Value'
        )
    )

In [5]:
norm_df = (
    pd.concat([
        read_sheet('Summary'),
        read_sheet('Student Achievement'),
        read_sheet('Framework'),
        read_sheet('Additional Info')],
        axis=0,
        ignore_index=True
    ).drop_duplicates()
)

In [6]:
school_types = norm_df[norm_df.Variable=='School Type']['Value'].unique()
for st in school_types:
    schools = norm_df.loc[(norm_df.Variable=='School Type') & (norm_df.Value==st), 'DBN'].values
    filt_df = (
        norm_df[norm_df.DBN.isin(schools)]
        .reset_index(drop=True)
        .pivot(index='DBN', columns='Variable', values='Value')
        .join(pd.DataFrame(school_directory).set_index('schooldbn'))
        .join(gdf)
        .reset_index()
        .set_index(['DBN', 'School Name'])
    )
    filt_df.drop(columns='geometry').to_csv(f'{st}_Schools_Data.csv')